<a href="https://colab.research.google.com/github/simnpeter/usernameApproving/blob/main/approveApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch pyngrok bitsandbytes accelerate -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.


In [ ]:
from flask import Flask, request, jsonify
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import accelerate
import torch
import threading
from pyngrok import ngrok
from google.colab import drive

drive.mount('/content/drive')

def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)

  generated_ids = model.generate(**encoded_input, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  response = decoded_output[0].replace(prompt, "")
  answer = response.split('\n')[-1:]
  answer = answer[-1].replace(".</s>", "").replace("</s>", "").strip()
  answer = answer[0].upper() + answer[1:]
  return answer

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)


# Define the path to the saved model
saved_model_path = "/content/drive/My Drive/Colab Notebooks/checkpoint-680"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(saved_model_path,
                                             device_map='auto',
                                             quantization_config=nf4_config,
                                             use_cache=False)
tokenizer = AutoTokenizer.from_pretrained(saved_model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
app = Flask(__name__)

ngrok.set_auth_token("2hyc71RAL73SYVIAFzwRLjibB18_468ZN3txG7uBYrv1DZxe6")
public_url =  ngrok.connect(500).public_url


@app.route('/predict', methods=['GET'])
def predict():
    #data = request.get_json()
    #name = data['name']

    #X = name
    while(1):
      predict = generate_response("<s>### Instruction:\nIn the following prompt, you will be given a username. Please determine if it is approvable by responding with either 'approved' or 'not approved'.Rules:\n- No personal info (e.g., last names or locations).\n- Use fictional names, not real people.\n- Avoid personal relationships (e.g., 'Mom' or 'Cousin').\n- No romantic terms like 'Wife' or 'Boyfriend'.\n- Avoid strong language or inappropriate references.\n- No political figures, parties, or institutions.\n- No references to illnesses, disorders, or death.\n- Ensure all media references are family-friendly.\n- No religious symbols, terms, or names.\n- Avoid using names of real celebrities, bands, teams, or brands.\n- Do not impersonate game developers or officials.\n- Avoid names similar to those of Cogs.\n- No complete sentences as names.\n- Use numbers and punctuation appropriately.\n- Ensure correct spelling and avoid typos.\n- Avoid nonsensical or repetitive text.\n- Ensure names contain vowels in correct positions.\n- Names should be under four words and not blank.\n- Use only alphabetic characters.\n- Merged words must be 15 characters or fewer and only contain two words.\n- Use simple and appropriate foreign language names.\n- Do not include URLs or web addresses.\n- Avoid creative alterations of the rules.\n\n### Input:\nfuckyou\n\n### Response:", model)
      if predict == 'Approved':
        approve = True
        break
      elif predict == 'Not approved':
        approve = False
        break


    return jsonify({"approved": approve}), 200


def start_server():
    app.run(host='0.0.0.0', port=5000)

threading.Thread(target=start_server).start()

# Start Ngrok tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

import time
time.sleep(10**6)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Public URL: NgrokTunnel: "https://bf91-34-143-199-81.ngrok-free.app" -> "http://localhost:5000"


INFO:werkzeug:127.0.0.1 - - [17/Jun/2024 15:36:40] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2024 15:36:40] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2024 15:36:50] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2024 15:41:31] "GET /robots.txt HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2024 15:41:34] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2024 15:43:03] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2024 15:43:04] "GET /favicon.ico HTTP/1.1" 404 -


KeyboardInterrupt: 